In [14]:
import pandas as pd
import polars as pl
import subprocess

In [12]:
%ls /home/jovyan/data/

'Cuenta Corriente PESOS 16-07-25.xlsx'*
'Cuenta Corriente PESOS 30-03-25.xlsx'*
'Cuenta Corriente PESOS 30-06-25.xlsx'*


In [15]:
# hago un ls de /home/jovyan/data/, me quedo todos los nomrbes de los archivos en un lista
cmd = "ls /home/jovyan/data/"
result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
file_names = result.stdout.splitlines()
print(file_names)

['Cuenta Corriente PESOS 16-07-25.xlsx', 'Cuenta Corriente PESOS 30-03-25.xlsx', 'Cuenta Corriente PESOS 30-06-25.xlsx']


In [17]:
import re

# ordeno ['Cuenta Corriente PESOS 16-07-25.xlsx', 'Cuenta Corriente PESOS 30-03-25.xlsx', 'Cuenta Corriente PESOS 30-06-25.xlsx'] por fecha
def extract_date(filename):
	match = re.search(r'(\d{2}-\d{2}-\d{2})', filename)
	if match:
		return pd.to_datetime(match.group(1), format='%d-%m-%y')
	else:
		return pd.NaT

file_names.sort(key=extract_date)
print(file_names)

['Cuenta Corriente PESOS 30-03-25.xlsx', 'Cuenta Corriente PESOS 30-06-25.xlsx', 'Cuenta Corriente PESOS 16-07-25.xlsx']


In [ ]:
min_date = '1900-01-01'

df_cuenta_corriente_historic = pd.DataFrame(columns=[
    'Liquida', 'Operado', 'Comprobante', 'Numero', 
    'Cantidad', 'Especie', 'Precio', 'Importe', 
    'Saldo', 'Referencia'
])

for file_name in file_names:
    data_dd_mm_yy = file_name.replace('Cuenta Corriente PESOS ', '').replace('.xlsx', '')
    df_cuenta_corriente = pd.read_excel(f'/home/jovyan/data/{file_name}', sheet_name=file_name.replace('.xlsx', ''))

    # filtro el df para quedarme con las fechas mayores a min_date
    df_cuenta_corriente['Operado'] = pd.to_datetime(df_cuenta_corriente['Operado'], format='%d/%m/%Y', errors='coerce')
    df_cuenta_corriente = df_cuenta_corriente[df_cuenta_corriente['Operado'] > min_date]
    # hago un union con el df_cuenta_corriente_historic
    df_cuenta_corriente_historic = pd.concat([df_cuenta_corriente_historic, df_cuenta_corriente], ignore_index=True)

    # el min_date se actualiza para que sea la maxima fecha de la columna Operado
    if not df_cuenta_corriente.empty:
        min_date = df_cuenta_corriente['Operado'].max()
    print(f'Procesado {file_name} con {len(df_cuenta_corriente)} filas')

df_cuenta_corriente_historic

Procesado Cuenta Corriente PESOS 30-03-25.xlsx con 185 filas
Procesado Cuenta Corriente PESOS 30-06-25.xlsx con 59 filas
Procesado Cuenta Corriente PESOS 16-07-25.xlsx con 17 filas


/tmp/ipykernel_254/1214895697.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_cuenta_corriente_historic = pd.concat([df_cuenta_corriente_historic, df_cuenta_corriente], ignore_index=True)


,Liquida,Operado,Comprobante,Numero,Cantidad,Especie,Precio,Importe,Saldo,Referencia
0,2024-04-09,2024-04-09,RECIBO DE COBRO,546744,0.00,NaN,0.00,"100,000.00","100,124.47",CREDITO CTA. CTE.
1,2024-04-09,2024-04-09,RECIBO DE COBRO,551326,0.00,NaN,0.00,200.00,"100,324.47",CREDITO CTA. CTE.
2,2024-04-10,2024-04-10,SUSCRIPCION FCI,120288,0.00,BULMAAA,0.00,"-80,000.00","20,324.47",NaN
3,2024-04-11,2024-04-09,VENTA,2910915,-10.00,ERIC,"2,662.18","26,621.84","46,946.31",NaN
4,2024-04-11,2024-04-09,COMPRA NORMAL,2918934,20.00,BPAT,"1,298.05","-25,960.92","20,985.39",NaN
...,...,...,...,...,...,...,...,...,...,...
256,2025-07-11,2025-07-01,VENTA CAUCION TERMINO,5062031,-100.00,VARIAS,755.35,"75,535.45","141,074.15",NaN
257,2025-07-11,2025-07-11,RETENCION,994819,0.00,NaN,0.00,-152.52,"140,921.63",RET GGAL
258,2025-07-11,2025-07-11,DIVIDENDOS,966184,0.00,GGAL,0.00,"2,149.84","143,071.47",GGAL BYMA
259,2025-07-14,2025-07-11,COMPRA NORMAL,5377333,250.00,FIPL,259.03,"-64,757.55","78,313.92",NaN


In [25]:
df_cuenta_corriente_historic.to_csv('/home/jovyan/data/cuenta_corriente_historic.csv', index=False)